## Check List 4 - MotionFilter
* Motion filters are quick decision makers that prevents trying to plan a infeasible motion.
* Here, ReachChecker is tested, compared with actual planning results in random trials
*\* ~~Strikethrough~~ means no-error but none-functional cases.*
* **4-A.2 ReachChecker**  
  - pick, place, ~~sweep~~
    - single process
    - multi process
    
* You need trained model to use ReachChecker
  - model/reach_svm/indy7.json
  - model/reach_svm/panda.json
  - 백업: 개인 이동식 하드디스크, 강준수

## set running directory to project source

In [1]:
import os
import numpy as np
import time
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

##### initialize CombinedRobot and GeometryScene

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *
from pkg.geometry.builder.scene_builder import SceneBuilder

s_builder = SceneBuilder(None)   # create scene builder without detector for virtual scene
crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, ((0,0,0), (0,0,0)), None)]
                     , connection_list=[False])
tip_link = crob.get_robot_tip_dict()[crob.robot_names[0]]
gscene = s_builder.create_gscene(crob)
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)
gscene.show_pose(crob.home_pose)

connection command:
indy0: False


Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker


##### add geometries - set collision=False for all, to make reach-only problems

In [3]:
from pkg.geometry.geometry import *

floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (3,3,0.01), (0,0,0), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=False)
wp1 = gscene.create_safe(GEOTYPE.BOX, "wp1", "base_link", (0.1,0.1,0.01), (0.5,-0.2,0),rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False, parent="floor")
wp2 = gscene.create_safe(GEOTYPE.BOX, "wp2", "base_link", (0.1,0.1,0.01), (0.5,0.2,0), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False, parent="floor")
goal = gscene.create_safe(GEOTYPE.BOX, "goal", "base_link", (0.1,0.1,0.01), (0.3,-0.4,0), 
                          rpy=(0,0,0), color=(0.2,0.2,0.8,1), display=True, fixed=True, collision=False)
 
# add movable (fixed=False for movable geometries)
box1 = gscene.create_safe(GEOTYPE.BOX, "box1", "base_link", (0.05,0.05,0.05), (0.3,0.4,0.031), 
                          rpy=(0,0,0), color=(0.8,0.2,0.2,1), display=True, fixed=False, collision=False)

##### create PlanningScene

In [4]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

##### create_binder

In [5]:
from pkg.planning.constraint.constraint_actor import PlacePlane, Gripper2Tool, SweepTool

In [6]:
# create PlacePlane on geometry "floor" and "goal"
# when point is not set, the entire upper surface of the geometry becomes valid binding area.
# when point is set, the specific point becomes the only valid binding point.
pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane)
pscene.create_binder(bname="goal", gname="goal", _type=PlacePlane, point=(0,0,0.005))

# add collision boundary for gripper base
# - set link_name="indy0_tcp" to attach the geometry to end-effector link
# - it can be labeled as fixed=True, as it is "fixed" on the indy0_tcp link
gripper =  gscene.create_safe(GEOTYPE.BOX, "gripper", link_name=tip_link, 
                                dims=(0.02,0.1,0.1), center=(0,0,0.05), rpy=(0,0,0), 
                                color=(0.8,0.2,0.2,0.5), display=True, fixed=True, collision=True)

# add gripper fingers - By setting parent="gripper", the position of geometry can be set relative to the parent geometry
finger1 =  gscene.create_safe(GEOTYPE.BOX, "finger1", link_name=tip_link,
                              dims=(0.02,0.02,0.1), center=(0,0.06,0.07), rpy=(0,0,0), 
                              color=(0.8,0.2,0.2,0.5), display=True, fixed=True, collision=True, parent="gripper")
finger2 =  gscene.create_safe(GEOTYPE.BOX, "finger2", link_name=tip_link, 
                              dims=(0.02,0.02,0.1), center=(0,-0.06,0.07), rpy=(0,0,0), 
                              color=(0.8,0.2,0.2,0.5), display=True, fixed=True, collision=True, parent="gripper")

# create Gripper2Tool binder
# Gripper2Tool is a 2-finger gripper, which can rotate along z-direction.
# To align the z-direction with the 2 fingers, rotate by 90 degree along roll axis.
# The gripping point is (0,0,0.11) in local coordinate of "gripper" geometry
pscene.create_binder(bname="grip0", gname="gripper", _type=Gripper2Tool, point=(0,0,0.11), rpy=(-np.pi/2,0,0))

# Add virtual (no-collision) sweep face. the point is 0.2 m away from the "indy0_tcp" link
# To match the z-direction with the target surface, the geometry is rotated 180 degrees in pitch-axis.
sweep_face =  gscene.create_safe(GEOTYPE.BOX, "sweep_face", link_name=tip_link, 
                                dims=(0.05,0.05,0.001), center=(0,0,0.2), rpy=(0,np.pi,0), 
                                color=(0.2,0.2,0.8,0.1), display=True, fixed=True, collision=False)
                                 
# create SweepTool binder
pscene.create_binder(bname="sweep_face", gname="sweep_face", _type=SweepTool, point=(0,0,0), rpy=(0,0,0))

##### create_subject

In [7]:
from pkg.planning.constraint.constraint_subject import Grasp2Point, PlacePoint, SweepPoint
from pkg.planning.constraint.constraint_subject import CustomObject, SweepLineTask
## create box object with grasping points along positive & negative y-direction and placement point in the bottom face
box_obj = pscene.create_subject(oname="box1", gname="box1", _type=CustomObject, 
                             action_points_dict = {
                                 "handle1": Grasp2Point("handle1", box1, [0,0,0], [-np.pi/2,0,0]),
                                 "handle2": Grasp2Point("handle2", box1, [0,0,0], [np.pi/2,0,0]),
                                 "bottom": PlacePoint("bottom", box1, [0,0,-0.026], [0,0,0])})
## create sweep task with 2 waypoints
sweep = pscene.create_subject(oname="sweep", gname="floor", _type=SweepLineTask, 
                             action_points_dict = {"wp1": SweepPoint("wp1", wp1, [0,0,0.005], [0,0,0]),
                                                   "wp2": SweepPoint("wp2", wp2, [0,0,0.005], [0,0,0])}
                             )

##### prepare motion planner

In [8]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)

##### ReachChecker
- pick, place, ~~sweep~~

In [9]:
from pkg.planning.filtering.reach_filter import ReachChecker
from pkg.planning.constraint.constraint_common import calc_redundancy

In [10]:
# ReachChecker needs end_link_couple_dict, which is a dictionary of list of coupled link names
mfilter = ReachChecker(pscene)

##### pick & place scenarios
  - below cell will automatically generate random configurations and record the filtering algorithm and planning results

##### Test a few runs with visualization

In [11]:
VISUALIZE = True
result_dict = defaultdict(list)
N_TRIAL = 10

for i_count in range(N_TRIAL):
    box1.set_offset_tf(center=(np.random.rand(3)-0.5)*2, 
                       orientation_mat=Rot_rpy((np.random.rand(3)-0.5)*2*pi))
    goal.set_offset_tf(center=(np.random.rand(3)-0.5)*2, 
                       orientation_mat=Rot_rpy((np.random.rand(3)-0.5)*2*pi))
    scenario = "pick"
    initial_state = pscene.initialize_state(crob.home_pose)
    pscene.set_object_state(initial_state)
    initial_state = pscene.rebind_all([('box1', 'bottom', 'floor', 'floor')], crob.home_pose)
    
    obj_name, handle_name, actor_name, actor_geo = "box1", "handle1", "grip0", "gripper"
    actor = pscene.actor_dict[actor_name]
    obj = pscene.subject_dict[obj_name]
    handle = obj.action_points_dict[handle_name]
    binding_list = [(obj_name, handle_name, actor_name, actor_geo)]
    constrained = False
    from_state=initial_state
    Q_dict = list2dict(from_state.Q, gscene.joint_names)
    to_state = from_state.copy(pscene)
    to_state.set_binding_state(
        ((obj_name, handle_name, actor_name, actor_geo), 
         to_state.binding_state[1]), 
        pscene)
    redundancy_dict = {
        obj_name:{handle_name:{"w":np.random.rand()*2*np.pi}, 
                  actor_name:{"w":np.random.rand()*2*np.pi}}}
    redundancy_values = {(obj_name, handle.name): calc_redundancy(redundancy_dict[obj_name][handle_name], handle),
                         (obj_name, actor.name): calc_redundancy(redundancy_dict[obj_name][actor_name], actor)
                        }

    pscene.set_object_state(from_state)
    gscene.show_pose(from_state.Q)
    success_mfilter = mfilter.check(actor, obj, handle, redundancy_values, Q_dict,constrained)

    Traj, LastQ, error, success = mplan.plan_algorithm(
        from_state, to_state, binding_list=binding_list, redundancy_values=redundancy_values)

    result_dict[scenario].append((success_mfilter, success))
    if success:
        pscene.set_object_state(from_state)
        if VISUALIZE:
            gscene.show_motion(Traj, period=0.05)
        pick_state = pscene.rebind_all(binding_list, LastQ)
    print("{}/{}: Result {} for {}: {} / {}".format(i_count, N_TRIAL, "match" if success_mfilter==success else "mismatch", scenario, success_mfilter, success))
    # assert success_mfilter == success, "Failure: filter result not same with motion planning result"
    
    
    if not success:
        pscene.set_object_state(initial_state)
        continue

    scenario = "place"
    obj_name, handle_name, actor_name, actor_geo = "box1", "bottom", "goal", "goal"
    actor = pscene.actor_dict[actor_name]
    obj = pscene.subject_dict[obj_name]
    handle = obj.action_points_dict[handle_name]
    binding_list = [(obj_name, handle_name, actor_name, actor_geo)]
    constrained = False
    from_state=pick_state
    Q_dict = list2dict(from_state.Q, gscene.joint_names)
    to_state = from_state.copy(pscene)
    to_state.set_binding_state(
        ((obj_name, handle_name, actor_name, actor_geo), 
         to_state.binding_state[1]), 
        pscene)
    redundancy_dict = {
        obj_name:{handle_name:{"w":np.random.rand()*2*np.pi}, 
                  actor_name:{"w":np.random.rand()*2*np.pi}}}
    redundancy_values = {(obj_name, handle.name): calc_redundancy(redundancy_dict[obj_name][handle_name], handle),
                         (obj_name, actor.name): calc_redundancy(redundancy_dict[obj_name][actor_name], actor)
                        }

    pscene.set_object_state(from_state)
    gscene.show_pose(crob.home_pose)
    success_mfilter = mfilter.check(actor, obj, handle, redundancy_values, Q_dict,constrained)

    Traj, LastQ, error, success = mplan.plan_algorithm(
        from_state, to_state, binding_list=binding_list, redundancy_values=redundancy_values)

    result_dict[scenario].append((success_mfilter, success))
    if success:
        if VISUALIZE:
            gscene.show_motion(Traj, period=0.05)
    pscene.set_object_state(initial_state)
    print("{}/{}: Result {} for {}: {} / {}".format(i_count, N_TRIAL, "match" if success_mfilter==success else "mismatch", scenario, success_mfilter, success))
    # assert success_mfilter == success, "Failure: filter result not same with motion planning result"

try transition motion
transition motion tried: False
0/10: Result match for pick: False / False
try transition motion
transition motion tried: False
1/10: Result match for pick: False / False
try transition motion
transition motion tried: False
2/10: Result match for pick: False / False
try transition motion
transition motion tried: False
3/10: Result match for pick: False / False
try transition motion
transition motion tried: True
4/10: Result match for pick: True / True
try transition motion
transition motion tried: True
4/10: Result match for place: True / True
try transition motion
transition motion tried: False
5/10: Result match for pick: False / False
try transition motion
transition motion tried: True
6/10: Result match for pick: True / True
try transition motion
transition motion tried: True
6/10: Result match for place: True / True
try transition motion
transition motion tried: False
7/10: Result match for pick: False / False
try transition motion
transition motion tried: Fal

##### massively test without visualization

In [12]:
VISUALIZE = False
result_dict = defaultdict(list)
N_TRIAL = 1000

for i_count in range(N_TRIAL):
    box1.set_offset_tf(center=(np.random.rand(3)-0.5)*2, 
                       orientation_mat=Rot_rpy((np.random.rand(3)-0.5)*2*pi))
    goal.set_offset_tf(center=(np.random.rand(3)-0.5)*2, 
                       orientation_mat=Rot_rpy((np.random.rand(3)-0.5)*2*pi))
    scenario = "pick"
    initial_state = pscene.initialize_state(crob.home_pose)
    pscene.set_object_state(initial_state)
    initial_state = pscene.rebind_all([('box1', 'bottom', 'floor', 'floor')], crob.home_pose)
    
    obj_name, handle_name, actor_name, actor_geo = "box1", "handle1", "grip0", "gripper"
    actor = pscene.actor_dict[actor_name]
    obj = pscene.subject_dict[obj_name]
    handle = obj.action_points_dict[handle_name]
    binding_list = [(obj_name, handle_name, actor_name, actor_geo)]
    constrained = False
    from_state=initial_state
    Q_dict = list2dict(from_state.Q, gscene.joint_names)
    to_state = from_state.copy(pscene)
    to_state.set_binding_state(
        ((obj_name, handle_name, actor_name, actor_geo), 
         to_state.binding_state[1]), 
        pscene)
    redundancy_dict = {
        obj_name:{handle_name:{"w":np.random.rand()*2*np.pi}, 
                  actor_name:{"w":np.random.rand()*2*np.pi}}}
    redundancy_values = {(obj_name, handle.name): calc_redundancy(redundancy_dict[obj_name][handle_name], handle),
                         (obj_name, actor.name): calc_redundancy(redundancy_dict[obj_name][actor_name], actor)
                        }

    pscene.set_object_state(from_state)
    gscene.show_pose(from_state.Q)
    success_mfilter = mfilter.check(actor, obj, handle, redundancy_values, Q_dict,constrained)

    Traj, LastQ, error, success = mplan.plan_algorithm(
        from_state, to_state, binding_list=binding_list, redundancy_values=redundancy_values)

    result_dict[scenario].append((success_mfilter, success))
    if success:
        pscene.set_object_state(from_state)
        if VISUALIZE:
            gscene.show_motion(Traj, period=0.05)
        pick_state = pscene.rebind_all(binding_list, LastQ)
    print("{}/{}: Result {} for {}: {} / {}".format(i_count, N_TRIAL, "match" if success_mfilter==success else "mismatch", scenario, success_mfilter, success))
    # assert success_mfilter == success, "Failure: filter result not same with motion planning result"
    
    
    if not success:
        pscene.set_object_state(initial_state)
        continue

    scenario = "place"
    obj_name, handle_name, actor_name, actor_geo = "box1", "bottom", "goal", "goal"
    actor = pscene.actor_dict[actor_name]
    obj = pscene.subject_dict[obj_name]
    handle = obj.action_points_dict[handle_name]
    binding_list = [(obj_name, handle_name, actor_name, actor_geo)]
    constrained = False
    from_state=pick_state
    Q_dict = list2dict(from_state.Q, gscene.joint_names)
    to_state = from_state.copy(pscene)
    to_state.set_binding_state(
        ((obj_name, handle_name, actor_name, actor_geo), 
         to_state.binding_state[1]), 
        pscene)
    redundancy_dict = {
        obj_name:{handle_name:{"w":np.random.rand()*2*np.pi}, 
                  actor_name:{"w":np.random.rand()*2*np.pi}}}
    redundancy_values = {(obj_name, handle.name): calc_redundancy(redundancy_dict[obj_name][handle_name], handle),
                         (obj_name, actor.name): calc_redundancy(redundancy_dict[obj_name][actor_name], actor)
                        }

    pscene.set_object_state(from_state)
    gscene.show_pose(crob.home_pose)
    success_mfilter = mfilter.check(actor, obj, handle, redundancy_values, Q_dict,constrained)

    Traj, LastQ, error, success = mplan.plan_algorithm(
        from_state, to_state, binding_list=binding_list, redundancy_values=redundancy_values)

    result_dict[scenario].append((success_mfilter, success))
    if success:
        if VISUALIZE:
            gscene.show_motion(Traj, period=0.05)
    pscene.set_object_state(initial_state)
    print("{}/{}: Result {} for {}: {} / {}".format(i_count, N_TRIAL, "match" if success_mfilter==success else "mismatch", scenario, success_mfilter, success))
    # assert success_mfilter == success, "Failure: filter result not same with motion planning result"

try transition motion
transition motion tried: True
0/1000: Result match for pick: True / True
try transition motion
transition motion tried: False
0/1000: Result match for place: False / False
try transition motion
transition motion tried: False
1/1000: Result match for pick: False / False
try transition motion
transition motion tried: False
2/1000: Result match for pick: False / False
try transition motion
transition motion tried: False
3/1000: Result match for pick: False / False
try transition motion
transition motion tried: True
4/1000: Result match for pick: True / True
try transition motion
transition motion tried: False
4/1000: Result match for place: False / False
try transition motion
transition motion tried: True
5/1000: Result match for pick: True / True
try transition motion
transition motion tried: False
5/1000: Result match for place: False / False
try transition motion
transition motion tried: False
6/1000: Result match for pick: False / False
try transition motion
tran

try transition motion
transition motion tried: False
61/1000: Result match for pick: False / False
try transition motion
transition motion tried: False
62/1000: Result match for pick: False / False
try transition motion
transition motion tried: False
63/1000: Result match for pick: False / False
try transition motion
transition motion tried: False
64/1000: Result match for pick: False / False
try transition motion
transition motion tried: False
65/1000: Result match for pick: False / False
try transition motion
transition motion tried: True
66/1000: Result match for pick: True / True
try transition motion
transition motion tried: False
66/1000: Result match for place: False / False
try transition motion
transition motion tried: False
67/1000: Result match for pick: False / False
try transition motion
transition motion tried: False
68/1000: Result match for pick: False / False
try transition motion
transition motion tried: True
69/1000: Result match for pick: True / True
try transition 

transition motion tried: False
129/1000: Result match for pick: False / False
try transition motion
transition motion tried: True
130/1000: Result match for pick: True / True
try transition motion
transition motion tried: False
130/1000: Result match for place: False / False
try transition motion
transition motion tried: False
131/1000: Result match for pick: False / False
try transition motion
transition motion tried: False
132/1000: Result match for pick: False / False
try transition motion
transition motion tried: False
133/1000: Result match for pick: False / False
try transition motion
transition motion tried: False
134/1000: Result match for pick: False / False
try transition motion
transition motion tried: False
135/1000: Result match for pick: False / False
try transition motion
transition motion tried: False
136/1000: Result match for pick: False / False
try transition motion
transition motion tried: True
137/1000: Result match for pick: True / True
try transition motion
trans

try transition motion
transition motion tried: False
199/1000: Result match for pick: False / False
try transition motion
transition motion tried: True
200/1000: Result match for pick: True / True
try transition motion
transition motion tried: False
200/1000: Result match for place: False / False
try transition motion
transition motion tried: False
201/1000: Result match for pick: False / False
try transition motion
transition motion tried: True
202/1000: Result match for pick: True / True
try transition motion
transition motion tried: False
202/1000: Result match for place: False / False
try transition motion
transition motion tried: False
203/1000: Result match for pick: False / False
try transition motion
transition motion tried: False
204/1000: Result match for pick: False / False
try transition motion
transition motion tried: True
205/1000: Result match for pick: True / True
try transition motion
transition motion tried: True
205/1000: Result match for place: True / True
try trans

transition motion tried: False
263/1000: Result match for place: False / False
try transition motion
transition motion tried: False
264/1000: Result match for pick: False / False
try transition motion
transition motion tried: False
265/1000: Result match for pick: False / False
try transition motion
transition motion tried: True
266/1000: Result match for pick: True / True
try transition motion
transition motion tried: False
266/1000: Result match for place: False / False
try transition motion
transition motion tried: True
267/1000: Result match for pick: True / True
try transition motion
transition motion tried: True
267/1000: Result mismatch for place: False / True
try transition motion
transition motion tried: True
268/1000: Result match for pick: True / True
try transition motion
transition motion tried: False
268/1000: Result match for place: False / False
try transition motion
transition motion tried: False
269/1000: Result match for pick: False / False
try transition motion
tran

...

try transition motion
transition motion tried: True
721/1000: Result match for pick: True / True
try transition motion
transition motion tried: False
721/1000: Result match for place: False / False
try transition motion
transition motion tried: False
722/1000: Result match for pick: False / False
try transition motion
transition motion tried: False
723/1000: Result match for pick: False / False
try transition motion
transition motion tried: False
724/1000: Result match for pick: False / False
try transition motion
transition motion tried: False
725/1000: Result match for pick: False / False
try transition motion
transition motion tried: False
726/1000: Result match for pick: False / False
try transition motion
transition motion tried: False
727/1000: Result match for pick: False / False
try transition motion
transition motion tried: False
728/1000: Result match for pick: False / False
try transition motion
transition motion tried: True
729/1000: Result match for pick: True / True
try t

try transition motion
transition motion tried: False
788/1000: Result match for pick: False / False
try transition motion
transition motion tried: True
789/1000: Result match for pick: True / True
try transition motion
transition motion tried: False
789/1000: Result match for place: False / False
try transition motion
transition motion tried: False
790/1000: Result match for pick: False / False
try transition motion
transition motion tried: False
791/1000: Result match for pick: False / False
try transition motion
transition motion tried: False
792/1000: Result match for pick: False / False
try transition motion
transition motion tried: True
793/1000: Result match for pick: True / True
try transition motion
transition motion tried: True
793/1000: Result match for place: True / True
try transition motion
transition motion tried: False
794/1000: Result match for pick: False / False
try transition motion
transition motion tried: True
795/1000: Result match for pick: True / True
try transi

transition motion tried: False
854/1000: Result match for pick: False / False
try transition motion
transition motion tried: False
855/1000: Result match for pick: False / False
try transition motion
transition motion tried: False
856/1000: Result match for pick: False / False
try transition motion
transition motion tried: False
857/1000: Result match for pick: False / False
try transition motion
transition motion tried: True
858/1000: Result match for pick: True / True
try transition motion
transition motion tried: False
858/1000: Result match for place: False / False
try transition motion
transition motion tried: True
859/1000: Result match for pick: True / True
try transition motion
transition motion tried: False
859/1000: Result match for place: False / False
try transition motion
transition motion tried: False
860/1000: Result match for pick: False / False
try transition motion
transition motion tried: False
861/1000: Result match for pick: False / False
try transition motion
tran

921/1000: Result match for pick: False / False
try transition motion
transition motion tried: False
922/1000: Result match for pick: False / False
try transition motion
transition motion tried: True
923/1000: Result match for pick: True / True
try transition motion
transition motion tried: False
923/1000: Result match for place: False / False
try transition motion
transition motion tried: True
924/1000: Result match for pick: True / True
try transition motion
transition motion tried: False
924/1000: Result match for place: False / False
try transition motion
transition motion tried: False
925/1000: Result match for pick: False / False
try transition motion
transition motion tried: True
926/1000: Result match for pick: True / True
try transition motion
transition motion tried: True
926/1000: Result match for place: True / True
try transition motion
transition motion tried: False
927/1000: Result match for pick: False / False
try transition motion
transition motion tried: False
928/1000:

transition motion tried: False
988/1000: Result match for pick: False / False
try transition motion
transition motion tried: True
989/1000: Result match for pick: True / True
try transition motion
transition motion tried: True
989/1000: Result match for place: True / True
try transition motion
transition motion tried: True
990/1000: Result match for pick: True / True
try transition motion
transition motion tried: False
990/1000: Result match for place: False / False
try transition motion
transition motion tried: False
991/1000: Result match for pick: False / False
try transition motion
transition motion tried: False
992/1000: Result match for pick: False / False
try transition motion
transition motion tried: False
993/1000: Result match for pick: False / False
try transition motion
transition motion tried: False
994/1000: Result match for pick: False / False
try transition motion
transition motion tried: True
995/1000: Result match for pick: True / True
try transition motion
transition

In [13]:
ACCURACY_CUT = 95
print("Accuracy cutline: {} %".format(ACCURACY_CUT))
for k, v in result_dict.items():
    v_arr = np.array(v)
    suc_idx = np.where(v_arr[:, 1])[0]
    acc_val = np.round(np.mean(v_arr[:,0]==v_arr[:,1])*100, 1)
    acc_suc = np.round(np.mean(v_arr[suc_idx,0]==v_arr[suc_idx,1])*100, 1)
    print("="*60)
    print("{} accuracy: {} % / success accuracy: {} % / success count:{}/{}".format(k, acc_val, acc_suc, len(suc_idx), len(v_arr)))
    assert acc_val>ACCURACY_CUT and acc_suc>ACCURACY_CUT, \
            "Accuracy {}/{} is lower than {}. Consider debugging the algorithm.".format(acc_val, acc_suc, ACCURACY_CUT)
    print("""OK with "{}" scenario""".format(k))

Accuracy cutline: 95 %
place accuracy: 98.1 % / success accuracy: 97.1 % / success count:69/263
OK with "place" scenario
pick accuracy: 98.7 % / success accuracy: 98.9 % / success count:263/1000
OK with "pick" scenario


##### sweep scenarios: reaching first sweep point and sweeping to second waypoint
  - as constrained motion is not considered in ReachChecker, we remove constraint of SweepLineTask here

In [14]:
# remove constraints here
sweep.make_constraints = lambda binding_from, binding_to: []

##### test a few runs with visualization

In [15]:
VISUALIZE = True
result_dict = defaultdict(list)
N_TRIAL = 10

for i_count in range(N_TRIAL):
    floor.set_offset_tf(center=(np.random.rand(3)-0.5)*1, 
                       orientation_mat=Rot_rpy((np.random.rand(3)-0.5)*pi/2))
    initial_state = pscene.initialize_state(crob.home_pose)
    pscene.set_object_state(initial_state)
    initial_state = pscene.rebind_all([('box1', 'bottom', 'floor', 'floor')], crob.home_pose)

    scenario = "reach wp1"
    obj_name, handle_name, actor_name, actor_geo = "sweep", "wp1", "sweep_face", "sweep_face"
    actor = pscene.actor_dict[actor_name]
    obj = pscene.subject_dict[obj_name]
    handle = obj.action_points_dict[handle_name]
    binding_list = [(obj_name, handle_name, actor_name, actor_geo)]
    constrained = False

    from_state=initial_state.copy(pscene)
    from_state.Q = crob.home_pose
    Q_dict = list2dict(from_state.Q, gscene.joint_names)
    to_state = from_state.copy(pscene)
    to_state.set_binding_state(
        (to_state.binding_state[1], 
         (obj_name, handle_name, actor_name, actor_geo)), 
        pscene)
    redundancy_dict = {
        obj_name:{handle_name:{"w":np.random.rand()*2*np.pi}, 
                  actor_name:{"w":np.random.rand()*2*np.pi}}}
    redundancy_values = {(obj_name, handle.name): calc_redundancy(redundancy_dict[obj_name][handle_name], handle),
                         (obj_name, actor.name): calc_redundancy(redundancy_dict[obj_name][actor_name], actor)
                        }

    pscene.set_object_state(from_state)
    gscene.show_pose(from_state.Q)
    success_mfilter = mfilter.check(actor, obj, handle, redundancy_values, Q_dict,constrained)
    Traj, LastQ, error, success = mplan.plan_algorithm(
        from_state, to_state, binding_list=binding_list, redundancy_values=redundancy_values)
            
    result_dict[scenario].append((success_mfilter, success))
    if success:
        pscene.set_object_state(from_state)
        if VISUALIZE:
            gscene.show_motion(Traj, period=0.05)
        sweep1_state = pscene.rebind_all(binding_list, LastQ)
    print("{}/{}: Result {} for {}: {} / {}".format(i_count, N_TRIAL, "match" if success_mfilter==success else "mismatch", scenario, success_mfilter, success))
#     assert success_mfilter == success, "Failure: filter result not same with motion planning result"

    if not success:
        pscene.set_object_state(initial_state)
        continue
        
    scenario = "sweep to wp2"
    obj_name, handle_name, actor_name, actor_geo = "sweep", "wp2", "sweep_face", "sweep_face"
    actor = pscene.actor_dict[actor_name]
    obj = pscene.subject_dict[obj_name]
    handle = obj.action_points_dict[handle_name]
    binding_list = [(obj_name, handle_name, actor_name, actor_geo)]
    constrained = True     # by setting constrained = True, although no effect in ReachChecker

    from_state=sweep1_state.copy(pscene)
    Q_dict = list2dict(from_state.Q, gscene.joint_names)
    to_state = from_state.copy(pscene)
    to_state.set_binding_state(
        (to_state.binding_state[1], 
         (obj_name, handle_name, actor_name, actor_geo)), 
        pscene)
    redundancy_dict = {
        obj_name:{handle_name:{"w":np.random.rand()*2*np.pi}, 
                  actor_name:{"w":np.random.rand()*2*np.pi}}}
    redundancy_values = {(obj_name, handle.name): calc_redundancy(redundancy_dict[obj_name][handle_name], handle),
                         (obj_name, actor.name): calc_redundancy(redundancy_dict[obj_name][actor_name], actor)
                        }

    pscene.set_object_state(from_state)
    gscene.show_pose(from_state.Q)
    success_mfilter = mfilter.check(actor, obj, handle, redundancy_values, Q_dict,constrained)
    Traj, LastQ, error, success = mplan.plan_algorithm(
        from_state, to_state, binding_list=binding_list, redundancy_values=redundancy_values,
        timeout=1)
    
    result_dict[scenario].append((success_mfilter, success))
    if success:
        if VISUALIZE:
            gscene.show_motion(Traj, period=0.05)
    print("{}/{}: Result {} for {}: {} / {}".format(i_count, N_TRIAL, "match" if success_mfilter==success else "mismatch", scenario, success_mfilter, success))
#     assert success_mfilter == success, "Failure: filter result not same with motion planning result"

try transition motion
transition motion tried: False
0/10: Result match for reach wp1: False / False
try transition motion
transition motion tried: False
1/10: Result match for reach wp1: False / False
try transition motion
transition motion tried: True
2/10: Result match for reach wp1: True / True
try transition motion
transition motion tried: True
2/10: Result match for sweep to wp2: True / True
try transition motion
transition motion tried: True
3/10: Result match for reach wp1: True / True
try transition motion
transition motion tried: True
3/10: Result match for sweep to wp2: True / True
try transition motion
transition motion tried: False
4/10: Result match for reach wp1: False / False
try transition motion
transition motion tried: True
5/10: Result match for reach wp1: True / True
try transition motion
transition motion tried: True
5/10: Result match for sweep to wp2: True / True
try transition motion
transition motion tried: False
6/10: Result match for reach wp1: False / False

##### massively test without visualization

In [16]:
VISUALIZE = False
result_dict = defaultdict(list)
N_TRIAL = 1000

for i_count in range(N_TRIAL):
    floor.set_offset_tf(center=(np.random.rand(3)-0.5)*1, 
                       orientation_mat=Rot_rpy((np.random.rand(3)-0.5)*pi/2))
    initial_state = pscene.initialize_state(crob.home_pose)
    pscene.set_object_state(initial_state)
    initial_state = pscene.rebind_all([('box1', 'bottom', 'floor', 'floor')], crob.home_pose)

    scenario = "reach wp1"
    obj_name, handle_name, actor_name, actor_geo = "sweep", "wp1", "sweep_face", "sweep_face"
    actor = pscene.actor_dict[actor_name]
    obj = pscene.subject_dict[obj_name]
    handle = obj.action_points_dict[handle_name]
    binding_list = [(obj_name, handle_name, actor_name, actor_geo)]
    constrained = False

    from_state=initial_state.copy(pscene)
    from_state.Q = crob.home_pose
    Q_dict = list2dict(from_state.Q, gscene.joint_names)
    to_state = from_state.copy(pscene)
    to_state.set_binding_state(
        (to_state.binding_state[1], 
         (obj_name, handle_name, actor_name, actor_geo)), 
        pscene)
    redundancy_dict = {
        obj_name:{handle_name:{"w":np.random.rand()*2*np.pi}, 
                  actor_name:{"w":np.random.rand()*2*np.pi}}}
    redundancy_values = {(obj_name, handle.name): calc_redundancy(redundancy_dict[obj_name][handle_name], handle),
                         (obj_name, actor.name): calc_redundancy(redundancy_dict[obj_name][actor_name], actor)
                        }

    pscene.set_object_state(from_state)
    gscene.show_pose(from_state.Q)
    success_mfilter = mfilter.check(actor, obj, handle, redundancy_values, Q_dict,constrained)
    Traj, LastQ, error, success = mplan.plan_algorithm(
        from_state, to_state, binding_list=binding_list, redundancy_values=redundancy_values)
            
    result_dict[scenario].append((success_mfilter, success))
    if success:
        pscene.set_object_state(from_state)
        if VISUALIZE:
            gscene.show_motion(Traj, period=0.05)
        sweep1_state = pscene.rebind_all(binding_list, LastQ)
    print("{}/{}: Result {} for {}: {} / {}".format(i_count, N_TRIAL, "match" if success_mfilter==success else "mismatch", scenario, success_mfilter, success))
#     assert success_mfilter == success, "Failure: filter result not same with motion planning result"

    if not success:
        pscene.set_object_state(initial_state)
        continue
        
    scenario = "sweep to wp2"
    obj_name, handle_name, actor_name, actor_geo = "sweep", "wp2", "sweep_face", "sweep_face"
    actor = pscene.actor_dict[actor_name]
    obj = pscene.subject_dict[obj_name]
    handle = obj.action_points_dict[handle_name]
    binding_list = [(obj_name, handle_name, actor_name, actor_geo)]
    constrained = True     # by setting constrained = True, although no effect in ReachChecker

    from_state=sweep1_state.copy(pscene)
    Q_dict = list2dict(from_state.Q, gscene.joint_names)
    to_state = from_state.copy(pscene)
    to_state.set_binding_state(
        (to_state.binding_state[1], 
         (obj_name, handle_name, actor_name, actor_geo)), 
        pscene)
    redundancy_dict = {
        obj_name:{handle_name:{"w":np.random.rand()*2*np.pi}, 
                  actor_name:{"w":np.random.rand()*2*np.pi}}}
    redundancy_values = {(obj_name, handle.name): calc_redundancy(redundancy_dict[obj_name][handle_name], handle),
                         (obj_name, actor.name): calc_redundancy(redundancy_dict[obj_name][actor_name], actor)
                        }

    pscene.set_object_state(from_state)
    gscene.show_pose(from_state.Q)
    success_mfilter = mfilter.check(actor, obj, handle, redundancy_values, Q_dict,constrained)
    Traj, LastQ, error, success = mplan.plan_algorithm(
        from_state, to_state, binding_list=binding_list, redundancy_values=redundancy_values,
        timeout=1)
    
    result_dict[scenario].append((success_mfilter, success))
    if success:
        if VISUALIZE:
            gscene.show_motion(Traj, period=0.05)
    print("{}/{}: Result {} for {}: {} / {}".format(i_count, N_TRIAL, "match" if success_mfilter==success else "mismatch", scenario, success_mfilter, success))
#     assert success_mfilter == success, "Failure: filter result not same with motion planning result"

try transition motion
transition motion tried: False
0/1000: Result match for reach wp1: False / False
try transition motion
transition motion tried: True
1/1000: Result match for reach wp1: True / True
try transition motion
transition motion tried: True
1/1000: Result match for sweep to wp2: True / True
try transition motion
transition motion tried: True
2/1000: Result match for reach wp1: True / True
try transition motion
transition motion tried: False
2/1000: Result match for sweep to wp2: False / False
try transition motion
transition motion tried: False
3/1000: Result match for reach wp1: False / False
try transition motion
transition motion tried: True
4/1000: Result match for reach wp1: True / True
try transition motion
transition motion tried: False
4/1000: Result match for sweep to wp2: False / False
try transition motion
transition motion tried: True
5/1000: Result match for reach wp1: True / True
try transition motion
transition motion tried: True
5/1000: Result match for sw

try transition motion
transition motion tried: False
57/1000: Result match for reach wp1: False / False
try transition motion
transition motion tried: True
58/1000: Result match for reach wp1: True / True
try transition motion
transition motion tried: True
58/1000: Result match for sweep to wp2: True / True
try transition motion
transition motion tried: True
59/1000: Result match for reach wp1: True / True
try transition motion
transition motion tried: True
59/1000: Result match for sweep to wp2: True / True
try transition motion
transition motion tried: True
60/1000: Result match for reach wp1: True / True
try transition motion
transition motion tried: False
60/1000: Result match for sweep to wp2: False / False
try transition motion
transition motion tried: False
61/1000: Result match for reach wp1: False / False
try transition motion
transition motion tried: True
62/1000: Result match for reach wp1: True / True
try transition motion
transition motion tried: True
62/1000: Result match

transition motion tried: False
111/1000: Result match for reach wp1: False / False
try transition motion
transition motion tried: True
112/1000: Result match for reach wp1: True / True
try transition motion
transition motion tried: True
112/1000: Result match for sweep to wp2: True / True
try transition motion
transition motion tried: True
113/1000: Result match for reach wp1: True / True
try transition motion
transition motion tried: True
113/1000: Result match for sweep to wp2: True / True
try transition motion
transition motion tried: False
114/1000: Result match for reach wp1: False / False
try transition motion
transition motion tried: False
115/1000: Result match for reach wp1: False / False
try transition motion
transition motion tried: True
116/1000: Result match for reach wp1: True / True
try transition motion
transition motion tried: True
116/1000: Result match for sweep to wp2: True / True
try transition motion
transition motion tried: True
117/1000: Result match for reach w

transition motion tried: False
158/1000: Result match for sweep to wp2: False / False
try transition motion
transition motion tried: True
159/1000: Result match for reach wp1: True / True
try transition motion
transition motion tried: True
159/1000: Result match for sweep to wp2: True / True
try transition motion
transition motion tried: False
160/1000: Result match for reach wp1: False / False
try transition motion
transition motion tried: False
161/1000: Result match for reach wp1: False / False
try transition motion
transition motion tried: False
162/1000: Result match for reach wp1: False / False
try transition motion
transition motion tried: False
163/1000: Result mismatch for reach wp1: True / False
try transition motion
transition motion tried: False
164/1000: Result match for reach wp1: False / False
try transition motion
transition motion tried: True
165/1000: Result match for reach wp1: True / True
try transition motion
transition motion tried: True
165/1000: Result match for

try transition motion
transition motion tried: True
213/1000: Result match for reach wp1: True / True
try transition motion
transition motion tried: True
213/1000: Result match for sweep to wp2: True / True
try transition motion
transition motion tried: True
214/1000: Result match for reach wp1: True / True
try transition motion
transition motion tried: True
214/1000: Result match for sweep to wp2: True / True
try transition motion
transition motion tried: True
215/1000: Result match for reach wp1: True / True
try transition motion
transition motion tried: True
215/1000: Result match for sweep to wp2: True / True
try transition motion
transition motion tried: False
216/1000: Result match for reach wp1: False / False
try transition motion
transition motion tried: True
217/1000: Result match for reach wp1: True / True
try transition motion
transition motion tried: True
217/1000: Result match for sweep to wp2: True / True
try transition motion
transition motion tried: True
218/1000: Resul

...

try transition motion
transition motion tried: True
765/1000: Result match for reach wp1: True / True
try transition motion
transition motion tried: False
765/1000: Result match for sweep to wp2: False / False
try transition motion
transition motion tried: True
766/1000: Result match for reach wp1: True / True
try transition motion
transition motion tried: True
766/1000: Result match for sweep to wp2: True / True
try transition motion
transition motion tried: True
767/1000: Result match for reach wp1: True / True
try transition motion
transition motion tried: True
767/1000: Result match for sweep to wp2: True / True
try transition motion
transition motion tried: True
768/1000: Result match for reach wp1: True / True
try transition motion
transition motion tried: True
768/1000: Result match for sweep to wp2: True / True
try transition motion
transition motion tried: False
769/1000: Result match for reach wp1: False / False
try transition motion
transition motion tried: False
770/1000: R

try transition motion
transition motion tried: False
816/1000: Result match for reach wp1: False / False
try transition motion
transition motion tried: False
817/1000: Result match for reach wp1: False / False
try transition motion
transition motion tried: True
818/1000: Result match for reach wp1: True / True
try transition motion
transition motion tried: True
818/1000: Result match for sweep to wp2: True / True
try transition motion
transition motion tried: False
819/1000: Result match for reach wp1: False / False
try transition motion
transition motion tried: True
820/1000: Result match for reach wp1: True / True
try transition motion
transition motion tried: True
820/1000: Result match for sweep to wp2: True / True
try transition motion
transition motion tried: False
821/1000: Result match for reach wp1: False / False
try transition motion
transition motion tried: True
822/1000: Result match for reach wp1: True / True
try transition motion
transition motion tried: True
822/1000: Re

try transition motion
transition motion tried: False
870/1000: Result match for reach wp1: False / False
try transition motion
transition motion tried: False
871/1000: Result match for reach wp1: False / False
try transition motion
transition motion tried: False
872/1000: Result mismatch for reach wp1: True / False
try transition motion
transition motion tried: True
873/1000: Result match for reach wp1: True / True
try transition motion
transition motion tried: True
873/1000: Result match for sweep to wp2: True / True
try transition motion
transition motion tried: False
874/1000: Result match for reach wp1: False / False
try transition motion
transition motion tried: False
875/1000: Result match for reach wp1: False / False
try transition motion
transition motion tried: True
876/1000: Result match for reach wp1: True / True
try transition motion
transition motion tried: True
876/1000: Result match for sweep to wp2: True / True
try transition motion
transition motion tried: True
877/100

923/1000: Result match for reach wp1: False / False
try transition motion
transition motion tried: True
924/1000: Result match for reach wp1: True / True
try transition motion
transition motion tried: False
924/1000: Result match for sweep to wp2: False / False
try transition motion
transition motion tried: True
925/1000: Result match for reach wp1: True / True
try transition motion
transition motion tried: True
925/1000: Result match for sweep to wp2: True / True
try transition motion
transition motion tried: True
926/1000: Result match for reach wp1: True / True
try transition motion
transition motion tried: True
926/1000: Result match for sweep to wp2: True / True
try transition motion
transition motion tried: True
927/1000: Result match for reach wp1: True / True
try transition motion
transition motion tried: False
927/1000: Result match for sweep to wp2: False / False
try transition motion
transition motion tried: True
928/1000: Result match for reach wp1: True / True
try transiti

try transition motion
transition motion tried: False
972/1000: Result match for reach wp1: False / False
try transition motion
transition motion tried: False
973/1000: Result match for reach wp1: False / False
try transition motion
transition motion tried: True
974/1000: Result match for reach wp1: True / True
try transition motion
transition motion tried: False
974/1000: Result match for sweep to wp2: False / False
try transition motion
transition motion tried: False
975/1000: Result match for reach wp1: False / False
try transition motion
transition motion tried: True
976/1000: Result match for reach wp1: True / True
try transition motion
transition motion tried: True
976/1000: Result match for sweep to wp2: True / True
try transition motion
transition motion tried: False
977/1000: Result match for reach wp1: False / False
try transition motion
transition motion tried: False
978/1000: Result match for reach wp1: False / False
try transition motion
transition motion tried: False
979/1

In [17]:
ACCURACY_CUT = 95
print("Accuracy cutline: {} %".format(ACCURACY_CUT))
for k, v in result_dict.items():
    v_arr = np.array(v)
    suc_idx = np.where(v_arr[:, 1])[0]
    acc_val = np.round(np.mean(v_arr[:,0]==v_arr[:,1])*100, 1)
    acc_suc = np.round(np.mean(v_arr[suc_idx,0]==v_arr[suc_idx,1])*100, 1)
    print("{} accuracy: {} % / success accuracy: {} % / success count:{}/{}".format(k, acc_val, acc_suc, len(suc_idx), len(v_arr)))
    assert acc_val>ACCURACY_CUT and acc_suc>ACCURACY_CUT, \
            "Accuracy {}/{} is lower than {}. Consider debugging the algorithm.".format(acc_val, acc_suc, ACCURACY_CUT)
    print("""OK with "{}" scenario""".format(k))

Accuracy cutline: 95 %
sweep to wp2 accuracy: 95.2 % / success accuracy: 98.6 % / success count:424/566
OK with "sweep to wp2" scenario
reach wp1 accuracy: 96.6 % / success accuracy: 99.8 % / success count:566/1000
OK with "reach wp1" scenario


##### planning: pick & place & sweep
- *NOTE*: We disabled all collision and motion constraint to check ReachChecker's functionality only.

In [18]:
from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)
ppline.set_motion_planner(MoveitPlanner(pscene, 
                                        motion_filters=[ReachChecker(pscene)]))
from pkg.planning.task.rrt import TaskRRT
ppline.set_task_planner(TaskRRT(pscene))

###### reset object positions

In [19]:
floor.set_offset_tf(center=(0,0,0), orientation_mat=Rot_rpy((0,0,0)))
goal.set_offset_tf(center=(0.3,-0.4,0), orientation_mat=Rot_rpy((0,0,0)))
box1.set_offset_tf(center=(0.3,0.4,0.031), orientation_mat=Rot_rpy((0,0,0)))
gscene.update_markers_all()

initial_state = pscene.initialize_state(crob.home_pose)
pscene.set_object_state(initial_state)

##### - single process

In [20]:
ppline.search(initial_state, goal_nodes=[("goal", 2)], verbose=True, 
              display=False, dt_vis=0.01, timeout_loop=100, multiprocess=False, timeout=1, timeout_constrained=10)
schedules = ppline.tplan.find_schedules()
schedules_sorted = ppline.tplan.sort_schedule(schedules)
ppline.play_schedule(ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0]), period=0.05)

try: 0 - ('floor', 0)->('floor', 1)
try transition motion
transition motion tried: True
result: 0 - ('floor', 0)->('floor', 1) = success
branching: 0->1 (0.02/100.0 s, steps/err: 13(18.7230110168 ms)/0.0017118649747)
try: 1 - ('floor', 1)->('floor', 2)
try transition motion
transition motion tried: True
result: 1 - ('floor', 1)->('floor', 2) = success
branching: 1->2 (0.04/100.0 s, steps/err: 12(18.1829929352 ms)/0.00131007490958)
try: 2 - ('floor', 2)->('gripper', 2)
try transition motion
transition motion tried: True
=============== try reaching goal from ('gripper', 2) =================
result: 2 - ('floor', 2)->('gripper', 2) = success
branching: 2->3 (0.06/100.0 s, steps/err: 10(15.7830715179 ms)/0.00154531086172)
try: 3 - ('gripper', 2)->('goal', 2)
try transition motion
transition motion tried: True
result: 3 - ('gripper', 2)->('goal', 2) = success
branching: 3->4 (0.09/100.0 s, steps/err: 6(37.5130176544 ms)/0.0015461110939)
++ adding return motion to acquired answer ++
try joi

##### - multi process

In [21]:
ppline.search(initial_state, goal_nodes=[("goal", 2)], verbose=True, 
              display=False, dt_vis=0.01, timeout_loop=100, multiprocess=True, timeout=1, timeout_constrained=10)
schedules = ppline.tplan.find_schedules()
schedules_sorted = ppline.tplan.sort_schedule(schedules)
ppline.play_schedule(ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0]), period=0.05)

Use 20/20 agents
try: 0 - ('floor', 0)->('gripper', 0)
try transition motion
try: 0 - ('floor', 0)->('floor', 1)
try transition motion
try: 0 - ('floor', 0)->('gripper', 0)
result: 0 - ('floor', 0)->('gripper', 0) = fail
try: 0 - ('floor', 0)->('floor', 1)
transition motion tried: True
transition motion tried: True
try transition motion
result: 0 - ('floor', 0)->('floor', 1) = success
result: 0 - ('floor', 0)->('gripper', 0) = success
branching: 0->2 (0.06/100.0 s, steps/err: 26(42.34790802 ms)/0.00109356880501)
branching: 0->1 (0.06/100.0 s, steps/err: 9(35.2809429169 ms)/0.00175671914036)
transition motion tried: True
try: 0 - ('floor', 0)->('gripper', 0)
try: 0 - ('floor', 0)->('floor', 1)
try transition motion
try transition motion
result: 0 - ('floor', 0)->('floor', 1) = success
branching: 0->3 (0.15/100.0 s, steps/err: 11(74.8450756073 ms)/0.00184380953231)
try: 2 - ('gripper', 0)->('goal', 0)
try transition motion
transition motion tried: True
transition motion tried: True
resul

try: 6 - ('goal', 0)->('gripper', 0)
transition motion tried: True
try: 22 - ('goal', 1)->('goal', 2)
result: 14 - ('goal', 1)->('goal', 2) = success
result: 6 - ('goal', 0)->('gripper', 0) = fail
branching: 14->23 (0.4/100.0 s, steps/err: 9(97.5608825684 ms)/0.00186957463903)
result: 14 - ('goal', 1)->('goal', 2) = success
=============== try reaching goal from ('goal', 1) =================
try: 11 - ('gripper', 0)->('gripper', 1)
transition motion tried: True
try transition motion
=============== try reaching goal from ('goal', 1) =================
result: 2 - ('gripper', 0)->('gripper', 1) = success
try: 6 - ('goal', 0)->('goal', 1)
++ adding return motion to acquired answer ++
transition motion tried: True
try joint motion
result: 6 - ('goal', 0)->('goal', 1) = success
branching: 2->25 (0.41/100.0 s, steps/err: 35(37.6110076904 ms)/0.00120742634495)
transition motion tried: True
try joint motion
try: 24 - ('goal', 1)->('goal', 2)
try transition motion
branching: 6->22 (0.41/100.0 s

branching: 34->46 (0.58/100.0 s, steps/err: 27(62.1039867401 ms)/0.00179754973048)
try: 40 - ('goal', 1)->('goal', 2)
======================= terminated 18: first answer acquired from other agent ===============================
result: 33 - ('goal', 0)->('goal', 1) = success
======================= terminated 7: first answer acquired from other agent ===============================
=============== try reaching goal from ('goal', 1) =================
result: 16 - ('gripper', 2)->('floor', 2) = success
++ adding return motion to acquired answer ++
======================= terminated 3: first answer acquired ===============================
++ adding return motion to acquired answer ++
branching: 16->48 (0.59/100.0 s, steps/err: 16(79.0438652039 ms)/0.00153376038316)
result: 33 - ('goal', 0)->('goal', 1) = success
branching: 33->45 (0.59/100.0 s, steps/err: 25(68.6409473419 ms)/0.00127286628776)
======================= terminated 6: first answer acquired from other agent ===================

Process Process-2:
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "pkg/planning/pipeline.py", line 152, in __search_loop
    display=display, dt_vis=dt_vis, **kwargs)
  File "pkg/planning/pipeline.py", line 200, in test_connection
    self.mplan.plan_transition(from_state, to_state, redundancy_dict=redundancy_dict, **kwargs)
  File "pkg/planning/motion/interface.py", line 105, in plan_transition
    redundancy_values=redundancy_values, **kwargs)
  File "pkg/planning/motion/moveit/moveit_planner.py", line 237, in plan_algorithm
    group_name, tool.geometry.link_name, goal_pose, target.geometry.link_name, tuple(from_Q), timeout=timeout)
  File "pkg/planning/motion/moveit/moveit_py.py", line 113, in plan_py
    JointState(self.joint_num, *Q_init), plannerconfig, timeout)
KeyboardInt